In [20]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, converters
from numpy import pi
from math import floor, log2

In [30]:
N = 15 # Number we want to factor
L = floor(log2(N) + 1)  # Number of bits needed to represent N
print(L)
epsilon = .25 # todo idk what this is
t = 2*L + 1 + floor(1 + log2(2 + 1/(2*epsilon)))  # formula from nielsen for required qbits
print(t)

4
12


In [22]:
# blatantly stole qft circuit generator from this:
# https://learn.qiskit.org/course/ch-algorithms/quantum-fourier-transform

def qft_rotations(circuit, n):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(pi/2**(n-qubit), qubit, n)
    # At the end of our function, we call the same function again on
    # the next qubits (we reduced n by one earlier in the function)
    qft_rotations(circuit, n)

def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft(circuit, n):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

def inverse_qft(circuit, n):
    """Does the inverse QFT on the first n qubits in circuit"""
    # First we create a QFT circuit of the correct size:
    qft_circ = qft(QuantumCircuit(n), n)
    # Then we take the inverse of this circuit
    invqft_circ = qft_circ.inverse()
    # And add it to the first n qubits in our existing circuit
    circuit.append(invqft_circ, circuit.qubits[:n])
    return circuit.decompose() # .decompose() allows us to see the individual gates

def qft_gate(n):
  qc = QuantumCircuit(n)
  qc.name = "QFT"
  qft(qc, n)
  gate = converters.circuit_to_gate(qc)

  return gate

def inverse_qft_gate(n):
  qc = QuantumCircuit(n)
  qc.name = "Inverse QFT"
  inverse_qft(qc, n)
  gate = converters.circuit_to_gate(qc)

  return gate

#fixme make this work for other numbers
def c_amod15_gate(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)
    if power % 2 != 0:
        if a in [2,13]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [7,8]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "y * %i^%i mod 15" % (a, power)

    # I guess you need to apply control more?
    c_U = U.control(t)
    return c_U

In [27]:
qreg1 = QuantumRegister(t, 'q')
qreg2 = QuantumRegister(L, 'qq')
creg1 = ClassicalRegister(t, 'c')

circuit_order_finding = QuantumCircuit(qreg1, qreg2, creg1)

for i in range(t):
  circuit_order_finding.reset(qreg1[i])

# t qbits in state |1>
# Init
for i in range(L):
  circuit_order_finding.reset(qreg2[i])
  circuit_order_finding.x(qreg2[i])

# Create superposition, modexp, inverse ft
circuit_order_finding.barrier(range(t+L))
#[circuit_order_finding.h(range(t)) for i in range(t)] bullshit
circuit_order_finding.h(range(t))
# basta che x sia co-primo con N
# for j in range(2**t): makes no sense
circuit_order_finding.append(c_amod15_gate(7, j), range(0, t+L)) #fixme I think this is the wrong algorithm...
circuit_order_finding.append(qft_gate(t).inverse(), range(0, t)) #todo inverse_qft_gate not working ??

#measure
circuit_order_finding.barrier(range(0, t+L))
circuit_order_finding.measure(range(t), range(t))


In [28]:
circuit_order_finding.draw()

░ ┌───┐                      ┌──────────┐ ░ ┌─┐               »
 q_0: ─|0>───────░─┤ H ├──────────■───────────┤0         ├─░─┤M├───────────────»
                 ░ ├───┤          │           │          │ ░ └╥┘┌─┐            »
 q_1: ─|0>───────░─┤ H ├──────────■───────────┤1         ├─░──╫─┤M├────────────»
                 ░ ├───┤          │           │          │ ░  ║ └╥┘┌─┐         »
 q_2: ─|0>───────░─┤ H ├──────────■───────────┤2         ├─░──╫──╫─┤M├─────────»
                 ░ ├───┤          │           │          │ ░  ║  ║ └╥┘┌─┐      »
 q_3: ─|0>───────░─┤ H ├──────────■───────────┤3         ├─░──╫──╫──╫─┤M├──────»
                 ░ ├───┤          │           │          │ ░  ║  ║  ║ └╥┘┌─┐   »
 q_4: ─|0>───────░─┤ H ├──────────■───────────┤4         ├─░──╫──╫──╫──╫─┤M├───»
                 ░ ├───┤          │           │          │ ░  ║  ║  ║  ║ └╥┘┌─┐»
 q_5: ─|0>───────░─┤ H ├──────────■───────────┤5         ├─░──╫──╫──╫──╫──╫─┤M├»
                 ░ ├───┤          │           │   QFT_dg │ ░  ║  ║  ║  ║  ║ └╥┘»
 q_6: ─|0>───────░─┤ H ├──────────■───────────┤6         ├─░──╫──╫──╫──╫──╫──╫─»
                 ░ ├───┤          │           │          │ ░  ║  ║  ║  ║  ║  ║ »
 q_7: ─|0>───────░─┤ H ├──────────■───────────┤7         ├─░──╫──╫──╫──╫──╫──╫─»
                 ░ ├───┤          │           │          │ ░  ║  ║  ║  ║  ║  ║ »
 q_8: ─|0>───────░─┤ H ├──────────■───────────┤8         ├─░──╫──╫──╫──╫──╫──╫─»
                 ░ ├───┤          │           │          │ ░  ║  ║  ║  ║  ║  ║ »
 q_9: ─|0>───────░─┤ H ├──────────■───────────┤9         ├─░──╫──╫──╫──╫──╫──╫─»
                 ░ ├───┤          │           │          │ ░  ║  ║  ║  ║  ║  ║ »
q_10: ─|0>───────░─┤ H ├──────────■───────────┤10        ├─░──╫──╫──╫──╫──╫──╫─»
                 ░ ├───┤          │           │          │ ░  ║  ║  ║  ║  ║  ║ »
q_11: ─|0>───────░─┤ H ├──────────■───────────┤11        ├─░──╫──╫──╫──╫──╫──╫─»
           ┌───┐ ░ └───┘┌─────────┴──────────┐└──────────┘ ░  ║  ║  ║  ║  ║  ║ »
qq_0: ─|0>─┤ X ├─░──────┤0                   ├─────────────░──╫──╫──╫──╫──╫──╫─»
           ├───┤ ░      │                    │             ░  ║  ║  ║  ║  ║  ║ »
qq_1: ─|0>─┤ X ├─░──────┤1                   ├─────────────░──╫──╫──╫──╫──╫──╫─»
           ├───┤ ░      │  y * 7^4095 mod 15 │             ░  ║  ║  ║  ║  ║  ║ »
qq_2: ─|0>─┤ X ├─░──────┤2                   ├─────────────░──╫──╫──╫──╫──╫──╫─»
           ├───┤ ░      │                    │             ░  ║  ║  ║  ║  ║  ║ »
qq_3: ─|0>─┤ X ├─░──────┤3                   ├─────────────░──╫──╫──╫──╫──╫──╫─»
           └───┘ ░      └────────────────────┘             ░  ║  ║  ║  ║  ║  ║ »
c: 12/════════════════════════════════════════════════════════╩══╩══╩══╩══╩══╩═»
                                                              0  1  2  3  4  5 »
«                        
« q_0: ──────────────────
«                        
« q_1: ──────────────────
«                        
« q_2: ──────────────────
«                        
« q_3: ──────────────────
«                        
« q_4: ──────────────────
«                        
« q_5: ──────────────────
«      ┌─┐               
« q_6: ┤M├───────────────
«      └╥┘┌─┐            
« q_7: ─╫─┤M├────────────
«       ║ └╥┘┌─┐         
« q_8: ─╫──╫─┤M├─────────
«       ║  ║ └╥┘┌─┐      
« q_9: ─╫──╫──╫─┤M├──────
«       ║  ║  ║ └╥┘┌─┐   
«q_10: ─╫──╫──╫──╫─┤M├───
«       ║  ║  ║  ║ └╥┘┌─┐
«q_11: ─╫──╫──╫──╫──╫─┤M├
«       ║  ║  ║  ║  ║ └╥┘
«qq_0: ─╫──╫──╫──╫──╫──╫─
«       ║  ║  ║  ║  ║  ║ 
«qq_1: ─╫──╫──╫──╫──╫──╫─
«       ║  ║  ║  ║  ║  ║ 
«qq_2: ─╫──╫──╫──╫──╫──╫─
«       ║  ║  ║  ║  ║  ║ 
«qq_3: ─╫──╫──╫──╫──╫──╫─
«       ║  ║  ║  ║  ║  ║ 
«c: 12/═╩══╩══╩══╩══╩══╩═
«       6  7  8  9  10 11